Import Libraries

In [1]:
import tensorflow as tf
from keras import layers, models

Load file and pahts

In [2]:
# Set your data paths and other parameters
train_data_path = "datasets/Training Images"
test_data_path = "datasets/Testing Images"

Data generator

In [3]:
# Image data generator for training
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(512, 512),  # Adjust the target size as needed
    batch_size=32,
    class_mode='binary',  # Use 'binary' for binary classification
    shuffle=True,
    seed=42
)

# Image data generator for testing (validation)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(512, 512),  # Adjust the target size as needed
    batch_size=32,
    class_mode='binary',  # Use 'binary' for binary classification
    shuffle=False  # No need to shuffle for testing
)

print(train_generator.class_indices)


Found 380 images belonging to 2 classes.


Found 20 images belonging to 2 classes.
{'Normal': 0, 'Not Normal': 1}


Build model

In [4]:
model = models.Sequential([
    layers.InputLayer(shape=(512, 512, 3)),  # Adjust input shape based on images
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification, so use 'sigmoid' activation
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 510, 510, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 255, 255, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 253, 253, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 126, 126, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 124, 124, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 62, 62, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 492032)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │   251,920,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 252,014,657 (961.36 MB)

 Trainable params: 252,014,657 (961.36 MB)

 Non-trainable params: 0 (0.00 B)

Train the Model (careful with epoc)

In [7]:
# Train the model
from keras.callbacks import EarlyStopping

# Define the early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', verbose=1)

# Train the model with early stopping callback
history = model.fit(train_generator, epochs=50, validation_data=test_generator, callbacks=[early_stopping])

Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 67s 5s/step - accuracy: 0.6751 - loss: 0.5925 - val_accuracy: 0.7000 - val_loss: 0.5026
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 67s 6s/step - accuracy: 0.6804 - loss: 0.5497 - val_accuracy: 0.6000 - val_loss: 0.5206
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 66s 5s/step - accuracy: 0.7113 - loss: 0.5487 - val_accuracy: 0.5500 - val_loss: 0.5879
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 67s 5s/step - accuracy: 0.6720 - loss: 0.5538 - val_accuracy: 0.7000 - val_loss: 0.4871
Epoch 5/50
 1/12 ━━━━━━━━━━━━━━━━━━━━ 1:07 6s/step - accuracy: 0.6250 - loss: 0.5917

KeyboardInterrupt: 

Evaluate Model

In [8]:
model.evaluate(test_generator)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 558ms/step - accuracy: 0.7000 - loss: 0.4949


[0.49491778016090393, 0.699999988079071]

Save Model

In [9]:
# Save the entire model to a HDF5 file
model.save("model70.h5")

print("Model Saved")

Model Saved
